In [1]:
import pandas as pd
import numpy as np

# --- STREAM 1: INTERNAL RETAIL DATA (Excel) ---
print("Step 1: Processing Internal Retail Data...")
file_path = 'online_retail_II.xlsx'

# 1. Load both years and combine
df_09_10 = pd.read_excel(file_path, sheet_name='Year 2009-2010')
df_10_11 = pd.read_excel(file_path, sheet_name='Year 2010-2011')
retail_df = pd.concat([df_09_10, df_10_11], ignore_index=True)

# 2. Clean column names for SQL
retail_df.columns = [col.lower().replace(' ', '_') for col in retail_df.columns]

# 3. Create necessary columns
retail_df['invoicedate'] = pd.to_datetime(retail_df['invoicedate'])
retail_df['total_value'] = retail_df['quantity'] * retail_df['price']
retail_df['is_return'] = retail_df['invoice'].astype(str).str.startswith('C')

# 4. FILTER FIRST (Create the 'uk_retail_clean' variable here)
uk_retail_clean = retail_df[(retail_df['country'] == 'United Kingdom') & (retail_df['price'] > 0)].copy()

# 5. CONVERT BOOLEAN TO INT (Now 'uk_retail_clean' exists!)
uk_retail_clean['is_return'] = uk_retail_clean['is_return'].astype(int)

# 6. EXPORT
uk_retail_clean.to_csv('cleaned_internal_sales_final.csv', index=False)

print("Step 1 Complete! File 'cleaned_internal_sales_final.csv' created.")

Step 1: Processing Internal Retail Data...
Step 1 Complete! File 'cleaned_internal_sales_final.csv' created.


In [2]:
# --- STREAM 2: EXTERNAL MARKET DATA (ONS CSV) ---
print("Step 2: Processing ONS Market Data...")
ons_raw = pd.read_csv('ons_retail_data.csv')

# ONS files are usually 'wide'. We need to find the "All Retailing" column.
# For this project, we want 'Year' and 'Value' (Current Prices).
# We filter for the period matching our internal data: 2009-2011
ons_filtered = ons_raw.copy() 

# Note: Depending on your specific ONS download, you may need to 
# rename the date column to 'report_date' and value to 'market_index'
# Example logic:
# ons_filtered = ons_filtered[['date', 'VALW']] 

Step 2: Processing ONS Market Data...


In [4]:
# --- STEP 3: EXPORTING FOR SQL ---
print("Step 3: Exporting cleaned files...")
uk_retail_clean.to_csv('cleaned_internal_sales.csv', index=False)
ons_filtered.to_csv('cleaned_external_market.csv', index=False)

print("Success! You now have two cleaned files ready for SQL matching.")

Step 3: Exporting cleaned files...
Success! You now have two cleaned files ready for SQL matching.


In [8]:

# Load the cleaned file
df = pd.read_csv('cleaned_internal_sales_final.csv')

print("--- DATA AUDIT REPORT ---")

# Check 1: Are there any non-UK countries?
non_uk = df[df['country'] != 'United Kingdom'].shape[0]
print(f"Non-UK rows remaining: {non_uk} (Should be 0)")

# Check 2: Are there any zero or negative prices?
bad_prices = df[df['price'] <= 0].shape[0]
print(f"Zero/Negative price rows: {bad_prices} (Should be 0)")

# Check 3: Did we handle the Returns correctly?
returns_count = df[df['is_return'] == True].shape[0]
print(f"Total Return transactions identified: {returns_count}")

# Check 4: Date Range Check (Ensuring 2009-2011 is covered)
print(f"Data starts on: {df['invoicedate'].min()}")
print(f"Data ends on: {df['invoicedate'].max()}")

--- DATA AUDIT REPORT ---
Non-UK rows remaining: 0 (Should be 0)
Zero/Negative price rows: 0 (Should be 0)
Total Return transactions identified: 16650
Data starts on: 2009-12-01 07:45:00
Data ends on: 2011-12-09 12:49:00


In [9]:
# Assuming your cleaned dataframe is named 'uk_retail_clean'
print("--- UK RETAIL DATA INTEGRITY REPORT ---")

# 1. Check for Boolean-to-Integer Conversion (For MySQL)
unique_returns = uk_retail_clean['is_return'].unique()
print(f"Values in 'is_return' column: {unique_returns}")
if set(unique_returns) <= {0, 1}:
    print("✅ SUCCESS: 'is_return' is correctly formatted as 0/1 for SQL.")
else:
    print("❌ ERROR: 'is_return' still contains non-numeric values.")

# 2. Check for Negative Prices (Financial Integrity)
negative_prices = uk_retail_clean[uk_retail_clean['price'] < 0].shape[0]
print(f"Transactions with negative prices: {negative_prices}")

# 3. Check for Date Consistency
start_date = uk_retail_clean['invoicedate'].min()
end_date = uk_retail_clean['invoicedate'].max()
print(f"Data Timeline: {start_date} to {end_date}")

# 4. Check for Column Nulls (Critical for SQL Import)
print("\n--- Missing Values Count ---")
print(uk_retail_clean.isnull().sum())

# 5. UK Market Logic Check: Top 5 Selling Products (by quantity)
print("\n--- Top 5 Best Sellers (UK) ---")
print(uk_retail_clean.groupby('description')['quantity'].sum().sort_values(ascending=False).head(5))

--- UK RETAIL DATA INTEGRITY REPORT ---
Values in 'is_return' column: [0 1]
✅ SUCCESS: 'is_return' is correctly formatted as 0/1 for SQL.
Transactions with negative prices: 0
Data Timeline: 2009-12-01 07:45:00 to 2011-12-09 12:49:00

--- Missing Values Count ---
invoice             0
stockcode           0
description         0
quantity            0
invoicedate         0
price               0
customer_id    233896
country             0
total_value         0
is_return           0
dtype: int64

--- Top 5 Best Sellers (UK) ---
description
WORLD WAR 2 GLIDERS ASSTD DESIGNS     99649
WHITE HANGING HEART T-LIGHT HOLDER    85440
ASSORTED COLOUR BIRD ORNAMENT         75526
JUMBO BAG RED RETROSPOT               71320
BROCADE RING PURSE                    70286
Name: quantity, dtype: int64
